In [2]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from scipy import stats
from matplotlib import cm

plt.rc('font', family='Malgun Gothic')
plt.style.use('ggplot')
plt.rcParams['axes.facecolor'] = '#f5f5f5'
warnings.filterwarnings('ignore')

colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']


In [3]:
df = pd.read_csv("train.csv", encoding="utf8")
df.iloc[0]["조식메뉴"]

'모닝롤/찐빵  우유/두유/주스 계란후라이  호두죽/쌀밥 (쌀:국내산) 된장찌개  쥐어채무침  포기김치 (배추,고추가루:국내산) '

In [4]:
df["조식메뉴"] = df["조식메뉴"].replace(to_replace=r'/', value=' ', regex=True) # 슬래쉬 ' '로 대체
df["조식메뉴"] = df["조식메뉴"].replace(r"\(.*\)", "", regex=True) # 괄호랑 괄호 안 내용 제거
df["조식메뉴"] = df["조식메뉴"].replace(r"  ", " ", regex=True) # 2칸 -> 1칸 whitespace
df["조식메뉴_배열"] = df["조식메뉴"].apply(lambda x: x[:-1].split())

In [31]:
df.iloc[1193:1203]["석식계"]

1193    451.0
1194    317.0
1195    360.0
1196    439.0
1197    277.0
1198    460.0
1199    414.0
1200    421.0
1201    353.0
1202    217.0
Name: 석식계, dtype: float64

In [32]:
df['일자'] = pd.to_datetime(df["일자"])
df['연도'] = df['일자'].dt.year

In [37]:
df.tail()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,조식메뉴_배열,연도
1200,2021-01-20,수,2983,75,198,4,391.0,모닝롤 페퍼로니피자 우유 주스 계란후라이 찐계란 크루통크림스프 흑미밥 아귀지리 마늘...,쌀밥/흑미밥/찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...,김치볶음밥 미니쫄우동*맛살튀김 브로콜리깨소스무침 계란후라이 고들빼기무침 겉절이김치,1093.0,421.0,"[모닝롤, 페퍼로니피자, 우유, 주스, 계란후라이, 찐계란, 크루통크림스프, 흑미밥...",2021
1201,2021-01-21,목,2983,92,231,462,351.0,모닝롤 생크림단팥빵 우유 주스 계란후라이 찐계란 누룽지탕 흑미밥 떡국 해물땡굴소스볶...,쌀밥/수수밥/찰현미밥 냉이된장국 동파육 봄동전 청경채/버섯숙회*초장 무생채 양상추샐...,흑미밥 쇠고기무국 삼치양념구이 비엔나채소볶음 숙주나물당근무침 포기김치,832.0,353.0,"[모닝롤, 생크림단팥빵, 우유, 주스, 계란후라이, 찐계란, 누룽지탕, 흑미밥, 떡...",2021
1202,2021-01-22,금,2983,255,248,1,303.0,모닝롤 BLT샌드위치 우유 주스 계란후라이 찐계란 흑임자죽 흑미밥 바지락살국 두부조...,전주비빔밥*약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...,흑미밥 수제비국 수제맛쵸킹탕수육 유부채소겨자냉채 참나물무침 갓김치/겉절이김치,579.0,217.0,"[모닝롤, BLT샌드위치, 우유, 주스, 계란후라이, 찐계란, 흑임자죽, 흑미밥, ...",2021
1203,2021-01-25,월,2983,107,153,616,327.0,모닝롤 호박고구마오븐구이 우유 주스 계란후라이 찐계란 누룽지탕 흑미밥 감자양파국 분...,쌀밥/흑미밥/찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부*달래양념장 봄...,흑미밥 열무된장국 장어강정*데리야끼소스 깻잎쌈*생강채 오이선 포기김치,1145.0,502.0,"[모닝롤, 호박고구마오븐구이, 우유, 주스, 계란후라이, 찐계란, 누룽지탕, 흑미밥...",2021
1204,2021-01-26,화,2983,69,183,551,362.0,모닝롤 야채샌드 우유 주스 계란후라이 찐계란 참치죽 흑미밥 홍합탕 애호박새우젓볶음 ...,쌀밥/귀리밥/찰현미밥 들깨미역국 교촌간장치킨 옥수수콘치즈구이 가지고추장무침 포기김치...,(New)할라피뇨멸치주먹밥 잔치국수 수제고기육전 쑥갓나물 양파초절임 깍두기,1015.0,480.0,"[모닝롤, 야채샌드, 우유, 주스, 계란후라이, 찐계란, 참치죽, 흑미밥, 홍합탕,...",2021


In [36]:
yearly = df.groupby("연도")["석식계"].mean()
yearly

연도
2016    519.418502
2017    457.049180
2018    463.539749
2019    445.386179
2020    430.668103
2021    396.588235
Name: 석식계, dtype: float64

In [26]:
feb = df[:18]
days = feb.groupby("요일")["본사휴가자수"].max()
feb.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,조식메뉴_배열,연도
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤 찐빵 우유 두유 주스 계란후라이 호두죽 쌀밥,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,"[모닝롤, 찐빵, 우유, 두유, 주스, 계란후라이, 호두죽, 쌀밥]",2016
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤 단호박샌드 우유 두유 주스 계란후라이 팥죽 쌀밥,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,"[모닝롤, 단호박샌드, 우유, 두유, 주스, 계란후라이, 팥죽, 쌀밥]",2016
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤 베이글 우유 두유 주스 계란후라이 표고버섯죽 쌀밥,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,"[모닝롤, 베이글, 우유, 두유, 주스, 계란후라이, 표고버섯죽, 쌀밥]",2016
3,2016-02-04,목,2601,104,220,355,0.0,모닝롤 토마토샌드 우유 두유 주스 계란후라이 닭죽 쌀밥,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,"[모닝롤, 토마토샌드, 우유, 두유, 주스, 계란후라이, 닭죽, 쌀밥]",2016
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤 와플 우유 두유 주스 계란후라이 쇠고기죽 쌀밥,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,"[모닝롤, 와플, 우유, 두유, 주스, 계란후라이, 쇠고기죽, 쌀밥]",2016


In [7]:
new_df = feb.explode("조식메뉴_배열")
# new_df.head()
new_df = pd.crosstab(
    new_df["조식메뉴_배열"], new_df["요일"]
).reset_index().rename_axis(columns=None)
new_df["총계"] = new_df[["금", "목", "수", "월", "화"]].sum(axis=1)
new_df.head()

,조식메뉴_배열,금,목,수,월,화,총계
0,견과류죽,0,1,0,0,0,1
1,계란후라이,4,5,3,4,3,19
2,고구마죽,1,0,0,0,0,1
3,김치죽,0,0,1,0,0,1
4,꿀호떡,0,0,1,0,0,1


## Procedures

0. 데이터 전처리 (D&D) : 알맞은 내용 채워넣기
1. 평균값 / 중앙값 계산 (입력)
2. 각 질문에 대한 대푯값 결정 (D&D)
3. 최댓값 / 평균값 (D&D)
4. 최댓값 (D&D)
5. 문제 해결 (선택)